<a href="https://colab.research.google.com/github/lisya/ML_in_business/blob/hw_final/1_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Notebooks/ML_in_business/final_project/Churn_Modelling.csv')
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Датасет содержит данные по клинтам банка.

Целевая переменная - удовлетворенность сервисом банка.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [4]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
RowNumber,10000,NaN,NaN,NaN,5000.5,2886.9,1,2500.75,5000.5,7500.25,10000
CustomerId,10000,NaN,NaN,NaN,1.56909e+07,71936.2,1.55657e+07,1.56285e+07,1.56907e+07,1.57532e+07,1.58157e+07
Surname,10000,2932,Smith,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CreditScore,10000,NaN,NaN,NaN,650.529,96.6533,350,584,652,718,850
Geography,10000,3,France,5014,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gender,10000,2,Male,5457,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,10000,NaN,NaN,NaN,38.9218,10.4878,18,32,37,44,92
Tenure,10000,NaN,NaN,NaN,5.0128,2.89217,0,3,5,7,10
Balance,10000,NaN,NaN,NaN,76485.9,62397.4,0,0,97198.5,127644,250898
NumOfProducts,10000,NaN,NaN,NaN,1.5302,0.581654,1,1,1,2,4


Пропусков нет. Каких-то явных выбросов тоже не видно.

Разобъем датасет на трейн и тест. Отбросим неинформативные признаки

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Exited','RowNumber', 'CustomerId', 'Surname']), 
                                                    df['Exited'], test_size=0.33, random_state=42)
#save test
X_test.to_csv("/content/drive/MyDrive/Notebooks/ML_in_business/final_project/X_test.csv", index=None)
y_test.to_csv("/content/drive/MyDrive/Notebooks/ML_in_business/final_project/y_test.csv", index=None)
#save train
X_train.to_csv("/content/drive/MyDrive/Notebooks/ML_in_business/final_project/X_train.csv", index=None)
y_train.to_csv("/content/drive/MyDrive/Notebooks/ML_in_business/final_project/y_train.csv", index=None)

Соберем пайплайн для обработки датасета

In [7]:
!pip install pdpipe

In [8]:
import pdpipe as pdp

Пайплайн получится совсем простой.Нам нужно будет только обработать категориальные признаки.

In [9]:
categorical_features = ['Geography', 'Gender']

In [10]:
features = pdp.OneHotEncode(categorical_features)
print(features)

PdPipelineStage: One-hot encode Geography, Gender


Проверим работу пайплайна.

In [11]:
features(X_train).head(5)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
8371,813,29,5,106059.40,1,0,0,187976.88,1,0,0
5027,681,37,8,73179.34,2,1,1,25292.53,1,0,1
9234,642,49,4,120688.61,1,1,0,24770.22,1,0,1
3944,612,31,8,117989.76,1,1,1,54129.86,0,0,0
6862,586,33,9,0.00,1,1,0,6975.02,0,0,1


In [12]:
from sklearn.pipeline import Pipeline

In [13]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

Будет истользовать XGB. Параметры подберем с помощью RandomizedSearchCV

In [14]:
cv_params={
    'learning_rate':[0.05,0.1,0.15,0.2,0.25,0.3],
    'max_depth':[3,4,5,6,8,10,12,15],
    'min_child_weight':[1,3,5,7],
    'gamma':[0.0,0.1,0.2,0.3,0.4],
    'colsample_bytree':[0.1,0.4,0.5,0.7]
}

In [15]:
clf=XGBClassifier()

In [16]:
random_search=RandomizedSearchCV(clf,param_distributions=cv_params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=1)

In [17]:
random_search.fit(features(X_train),y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    8.3s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=None,
                                           objective='binary:logistic',
                                           random_state=0, reg_alpha=0,
                                           reg_lambda=1, sc...
                                           verbosity=1),
                   iid='deprecated', n_iter=5, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0

In [18]:
best_params = random_search.best_params_
best_params

{'colsample_bytree': 0.7,
 'gamma': 0.3,
 'learning_rate': 0.15,
 'max_depth': 3,
 'min_child_weight': 5}

In [19]:
pipeline = Pipeline([
    ('features',features),
    ('classifier', XGBClassifier(**best_params))])

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features', PdPipelineStage: One-hot encode Geography, Gender),
                ('classifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=0.7, gamma=0.3,
                               learning_rate=0.15, max_delta_step=0,
                               max_depth=3, min_child_weight=5, missing=None,
                               n_estimators=100, n_jobs=1, nthread=None,
                               objective='binary:logistic', random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=None, silent=None, subsample=1,
                               verbosity=1))],
         verbose=False)

In [20]:
from sklearn.metrics import roc_auc_score,roc_curve

In [21]:
predictions = pipeline.predict_proba(X_test)

In [22]:
predictions[:, 1]

array([0.04591011, 0.04164004, 0.09128173, ..., 0.18483299, 0.15338588,
       0.76098096], dtype=float32)

In [23]:
roc_auc_score(y_score=predictions[:, 1], y_true=y_test)

0.8686093426150356

In [24]:
import dill

Сохраним модель 

In [25]:
with open("/content/drive/MyDrive/Notebooks/ML_in_business/final_project/xgb_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)